In [1]:
import xarray as xr
import geopandas as gpd
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [2]:
experiment_name = 'INALT60.L120-KRS0020'
data_resolution = '1d'

sigma = 9
params = 'OW0.2_Npix-600-15000' # 'OW0.5_Npix-1296-32400'

In [3]:
# defined in function below: outpath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/depth-{depth}/'
file_pattern = "Eddies_{time}_{params}.pickle"

In [4]:
#periods = ("2012-04-10", "2012-05-04")
periods = [
#    ("2012-01-01", "2012-01-25"), ("2012-01-26", "2012-02-19"), ("2012-02-20", "2012-03-15"), ("2012-03-16", "2012-04-09"),
    ("2012-04-10", "2012-05-04"),
#    ("2012-05-05", "2012-05-29"), ("2012-05-30", "2012-06-28"), ("2012-10-27", "2012-11-20"), ("2012-11-21", "2012-12-15"),   
#    ("2012-06-29", "2012-07-28"), ("2012-07-29", "2012-08-27"), ("2012-08-28", "2012-09-26"), ("2012-09-27", "2012-10-26"),
]

In [5]:
#depth_information = (0,0)

mesh_mask = xr.open_dataset(f'/gxfs_work/geomar/smomw355/model_data/ocean-only/{experiment_name}/nemo/suppl/2_{experiment_name}_mesh_mask.nc') 
depth_information = [(round(mesh_mask.nav_lev.values[i]), i) for i in range(0, 91, 3)] #upper 3000m, otherwise: range(0, len(mesh_mask.nav_lev.values), 3)
print(depth_information)

[(0, 0), (4, 3), (10, 6), (18, 9), (27, 12), (39, 15), (54, 18), (71, 21), (91, 24), (114, 27), (141, 30), (173, 33), (210, 36), (253, 39), (304, 42), (365, 45), (437, 48), (525, 51), (629, 54), (753, 57), (899, 60), (1066, 63), (1254, 66), (1461, 69), (1684, 72), (1922, 75), (2170, 78), (2428, 81), (2692, 84), (2963, 87), (3238, 90)]


In [6]:
lon = mesh_mask.nav_lon.data
lat = mesh_mask.nav_lat.data

In [7]:
def compute_OW_mask(start, end, depth):
    outpath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/depth-{depth}/'
    output_file = outpath + f'Parcels-mask_{start.replace("-", "")}_{end.replace("-", "")}_{params}.nc'
    if os.path.exists(output_file):
        print(f'{depth}m: {start.replace("-", "")} to {end.replace("-", "")} with {params} already exists. Skip ...')
        return

    time_str_list = []
    mask_list = []
    area_list = []
    type_list = []
    scale_list = []
    
    time_array = pd.date_range(start=start, end=end, freq=data_resolution).to_pydatetime()
    for time in time_array:
        time_str = time.strftime("%Y-%m-%d") #T%H
        time_str_list.append(time_str)
        file_path = outpath + file_pattern.format(time=time_str, params=params, depth=depth)
        with open(file_path, 'rb') as f:
            OW_1 = pickle.load(f)
        
        # empty fields
        mask = np.zeros_like(lon, dtype=int)
        area_field = np.zeros_like(lon, dtype=float)
        type_field = np.zeros_like(lon, dtype=int)
        scale_field = np.zeros_like(lon, dtype=float)
        
        # all eddies in OW_1
        for i in range(len(OW_1)):
            indices_i = OW_1[i]['eddy_i']
            indices_j = OW_1[i]['eddy_j']
            
            mask[indices_j, indices_i] = 1
            area_field[indices_j, indices_i] = OW_1[i]['area']
            scale_field[indices_j, indices_i] = OW_1[i]['scale']
            
            #type_field[indices_j, indices_i] = OW_1[i]['type']
            if OW_1[i]['type'] == 'anticyclonic':
                type_field[indices_j, indices_i] = -1
            elif OW_1[i]['type'] == 'cyclonic':
                type_field[indices_j, indices_i] = 1
            
        mask_list.append(mask)
        area_list.append(area_field)
        type_list.append(type_field)
        scale_list.append(scale_field)
        
    # Combine
    OW = xr.Dataset(
        {
            "eddymask": (["time_counter", "y", "x"], np.array(mask_list, dtype=np.int8)),
            "area": (["time_counter", "y", "x"], np.array(area_list, dtype=np.float32)),
            "type": (["time_counter", "y", "x"], np.array(type_list, dtype=np.int8)),
            "scale": (["time_counter", "y", "x"], np.array(scale_list, dtype=np.float32)),
        },
        coords={
            "time_counter": time_array,
            "nav_lon": (["x"], lon[0]),
            "nav_lat": (["y"], lat[:,0]),
        },
    ).chunk({"time_counter": 1, "y": 50, "x": 50})
    
    # Save
    OW.to_netcdf(outpath + f'Parcels-mask_{time_str_list[0].replace("-", "")}_{time_str_list[-1].replace("-", "")}_{params}.nc')

In [8]:
for datestart, dateend in periods:
    for (depth, depth_index) in tqdm(depth_information, desc='depth levels'):
        #print(f"> Processing period: {datestart} to {dateend}, {depth}m ...")
        compute_OW_mask(datestart, dateend, depth)

depth levels:   0%|          | 0/31 [00:00<?, ?it/s]

0m: 20120410 to 20120504 with OW0.2_Npix-600-15000 already exists. Skip ...
4m: 20120410 to 20120504 with OW0.2_Npix-600-15000 already exists. Skip ...
10m: 20120410 to 20120504 with OW0.2_Npix-600-15000 already exists. Skip ...
18m: 20120410 to 20120504 with OW0.2_Npix-600-15000 already exists. Skip ...
27m: 20120410 to 20120504 with OW0.2_Npix-600-15000 already exists. Skip ...
39m: 20120410 to 20120504 with OW0.2_Npix-600-15000 already exists. Skip ...
54m: 20120410 to 20120504 with OW0.2_Npix-600-15000 already exists. Skip ...
71m: 20120410 to 20120504 with OW0.2_Npix-600-15000 already exists. Skip ...
91m: 20120410 to 20120504 with OW0.2_Npix-600-15000 already exists. Skip ...
114m: 20120410 to 20120504 with OW0.2_Npix-600-15000 already exists. Skip ...
141m: 20120410 to 20120504 with OW0.2_Npix-600-15000 already exists. Skip ...
173m: 20120410 to 20120504 with OW0.2_Npix-600-15000 already exists. Skip ...
210m: 20120410 to 20120504 with OW0.2_Npix-600-15000 already exists. Skip .

depth levels: 100%|██████████| 31/31 [11:45<00:00, 22.74s/it]


In [9]:
combined_outpath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/'
combined_filename = f'Parcels-mask_depths--3_{datestart.replace("-", "")}_{dateend.replace("-", "")}_{params}.nc'

datasets = []

for datestart, dateend in periods:
    for (depth, depth_index) in tqdm(depth_information, desc='depth levels'):
        depth_outpath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/depth-{depth}/'
        depth_filename = f'Parcels-mask_{datestart.replace("-", "")}_{dateend.replace("-", "")}_{params}.nc'
        depth_filepath = os.path.join(depth_outpath, depth_filename)
        
        if os.path.exists(depth_filepath):
            ds = xr.open_dataset(depth_filepath)
            ds = ds.assign_coords(nav_lev=mesh_mask.nav_lev.values[depth_index]).expand_dims("z")
            datasets.append(ds)
        else:
            print(f"{depth_filepath} not found, skip ...")

if datasets:
    final_ds = xr.concat(datasets, dim="z") 
    final_ds.to_netcdf(os.path.join(combined_outpath, combined_filename))
    print(f"Saved: {combined_outpath + combined_filename}")
else:
    print("No files found.")


depth levels: 100%|██████████| 31/31 [00:24<00:00,  1.28it/s]


Saved: /gxfs_work/geomar/smomw523/eddytools/results/INALT60.L120-KRS0020/smoothed/9/1d/Parcels-mask_depths--3_20120410_20120504_OW0.2_Npix-600-15000.nc


In [10]:
final_ds

<xarray.Dataset>
Dimensions:       (z: 31, time_counter: 25, y: 1483, x: 2404)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2012-04-10 ... 2012-05-04
    nav_lon       (x) float32 0.05 0.06667 0.08333 0.1 ... 40.07 40.08 40.1
    nav_lat       (y) float32 -44.97 -44.95 -44.94 ... -24.95 -24.94 -24.92
    nav_lev       (z) float32 0.4768 4.336 9.996 ... 2.963e+03 3.238e+03
Dimensions without coordinates: z, y, x
Data variables:
    eddymask      (z, time_counter, y, x) int8 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    area          (z, time_counter, y, x) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    type          (z, time_counter, y, x) int8 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
    scale         (z, time_counter, y, x) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0